<a href="https://colab.research.google.com/github/Anurag17singh/Research_Agent/blob/main/Research_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community langchain_huggingface langgraph huggingface_hub sentence_transformers tavily-python transformers accelerate bitsandbytes sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [ ]:
from tavily import TavilyClient
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langgraph.graph import StateGraph
import torch
from typing import TypedDict

In [ ]:
from huggingface_hub import login
login()


In [ ]:
tavily = TavilyClient(api_key="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")

In [ ]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")

# Create a pipeline for text generation
llm_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
        return_full_text=False,
        max_new_tokens=300,
        temperature=0.3,
        do_sample=True,
)

Device set to use cuda:0


In [ ]:
llm = HuggingFacePipeline(pipeline=llm_pipeline)

<ipython-input-19-c7453ec2d9a3>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [ ]:
def research_agent(state):
    url = state["url"]
    question = state["question"]

    # Fetch content using Tavily
    result = tavily.extract(urls=[url])
    title = result.get("title", "No title")
    content = result.get("content", "No content available")[:5000]  # Truncate to avoid overloading

    return {
        "question": question,
        "research_summary": f"Title: {title}\n\nContent:\n{content}",
         "chat_history": state.get("chat_history", [])
    }

prompt = PromptTemplate(
    input_variables=["research_summary", "question", "chat_history_str"],
    template="""
You are a helpful research assistant.

Here is the chat history so far:
{chat_history_str}

Here is new information from the website:
{research_summary}

Now answer the user's question:
{question}
"""
)
answer_chain = LLMChain(llm=llm, prompt=prompt)

def answer_agent(state):
    chat_history = state.get("chat_history", [])

    # Convert past chat history to a nice readable string
    chat_history_str = "\n".join(
        [f"User: {item['question']}\nAssistant: {item['answer']}" for item in chat_history]
    )

    final_answer = answer_chain.run(
        research_summary=state["research_summary"],
        question=state["question"],
        chat_history_str=chat_history_str
    )

    # Update chat history
    chat_history.append({
        "question": state["question"],
        "answer": final_answer
    })

    return {
        "final_answer": final_answer,
        "chat_history": chat_history
    }

In [ ]:
from typing import TypedDict, List, Dict # Import List and Dict

class AgentState(TypedDict):
    url: str
    question: str
    research_summary: str
    final_answer: str
    chat_history: List[Dict[str, str]]  # Chat history (list of dicts with question-answer)

# ... (rest of your code)
class AgentState(TypedDict):
    url: str
    question: str
    research_summary: str
    final_answer: str
    chat_history: List[Dict[str, str]]  # Chat history (list of dicts with question-answer)

graph = StateGraph(AgentState)

graph.add_node("research", research_agent)
graph.add_node("answer", answer_agent)

graph.set_entry_point("research")
graph.add_edge("research", "answer")
graph.set_finish_point("answer")

compiled_graph = graph.compile()
chat_history = []

In [ ]:
while True:
    url = input("\n🌐 Enter Website URL (or 'exit' to quit): ")
    if url.lower() == "exit":
        break

    question = input("❓ Enter your question: ")

    input_data = {
        "url": url,
        "question": question,
        "chat_history": chat_history,
    }

    result = compiled_graph.invoke(input_data)

    # Update chat history
    chat_history = result["chat_history"]
    print("\n🧠 Assistant's Answer:")
    print(result["final_answer"])


🌐 Enter Website URL (or 'exit' to quit): https://en.wikipedia.org/wiki/Lionel_Messi
❓ Enter your question: Who is Lionel Messi?

🧠 Assistant's Answer:

Given text: "Lionel Messi is a professional footballer who plays as a forward for Spanish club Barcelona and the Argentina national team."

🌐 Enter Website URL (or 'exit' to quit): exit
